In [13]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns

In [14]:
df_transactions_wofirstname = pd.read_csv(x +
                                    "Thalia_transactions_ohne_vorname.csv")

# Datenbereinigung und Erstellung einer Privatkunden Dataframe

In [16]:
#Transaktions-Daten bereinigen

df_t = df_transactions_wofirstname[["iid", "key", "attrtype", "status", "pricetotal"]]
df_t = df_t.query('attrtype == "TI"')
df_t = df_t.query('status == "Buchung" or status == "Order"')
df_t = df_t.query('pricetotal >= 0.01')

# Spalte mit Mean-TIs erstellen
df_t = df_t.groupby(["iid", "key"])["key"].agg("count")
df_t = df_t.to_frame()
df_t = df_t.rename(columns = {"key" : "AnzahlTIsProOrder"})
df_t = df_t.reset_index()

df_t = df_t.groupby("iid")["AnzahlTIsProOrder"].agg("mean")
df_t = df_t.to_frame()
df_t = df_t.rename(columns = {"AnzahlTIsProOrder" : "MeanTIperOR"})
df_t = df_t.reset_index()
df_t.head()

,iid,MeanTIperOR
0,100002747,2.5
1,100009615,2.0
2,100009745,4.8
3,100011333,2.0
4,100011954,2.1


In [17]:
# Computing IQR
Q1 = df_t["MeanTIperOR"].quantile(0.25)
Q3 = df_t["MeanTIperOR"].quantile(0.75)
IQR = Q3 - Q1
IQR_neu = Q3 + 1.5 * IQR
print(IQR_neu)

3.25


In [18]:
#Ausgabe der iids die als Privatkunden klassifiziert werden

df_t = df_t.query("MeanTIperOR <= @IQR_neu")
df_t = df_t["iid"]

In [19]:
df_t

0         100002747
1         100009615
3         100011333
4         100011954
5         100013175
            ...    
145573    290128953
145574    290130157
145575    290130388
145576    290131783
145577    290131845
Name: iid, Length: 128623, dtype: int64

In [8]:
#Exportieren der Privatkunden iids

export_csv = df_t.to_csv('privatkunden.iid.thalia.csv')